>### Inception Network

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

#### Data

In [30]:
def preprocess(image, label): 
    image = tf.cast(image, tf.float32) 
    image = tf.image.resize(image, [299, 299]) #resize前默认图片dim=3
    c1 = tf.expand_dims(image[...,0],-1)*(0.229/0.5)+(0.485-0.5)/0.5
    c2 = tf.expand_dims(image[...,1],-1)*(0.224/0.5)+(0.456-0.5)/0.5
    c3 = tf.expand_dims(image[...,2],-1)*(0.225/0.5)+(0.406-0.5)/0.5
    image = tf.concat([c1, c2, c3], axis=-1)
    image = tf.clip_by_value(image,0,255)
    image = image/255
    return image, label 

#### InceptionV1
Paper: [Going Deeper with Convolutions](https://arxiv.org/abs/1409.4842)

<img src='https://miro.medium.com/max/1000/1*KnTe9YGNopUMiRjlEr3b8w.png'></img>

#### InceptionV3
Paper: [Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/abs/1512.00567)

<img src='https://miro.medium.com/max/1000/1*ooVUXW6BIcoRdsF7kzkMwQ.png'></img>

In [315]:
class ConvBlock(keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='valid', zeros=None): 
        super(ConvBlock, self).__init__()
        self.zeros = zeros
        if self.zeros:
            self.p = layers.ZeroPadding2D(zeros)
        self.conv = layers.Conv2D(filters, kernel_size, strides, padding=padding) 
        self.bn = layers.BatchNormalization()
        self.relu = layers.Activation('relu') 
        
    def call(self, x):
        if self.zeros:
            x = self.p(x)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [333]:
#不做下采样 (w,h)->(w,h)
class InceptionA(keras.Model): 
    def __init__(self, pool_features): 
        super(InceptionA, self).__init__()
        self.branch1x1 = ConvBlock(64, 1)
        self.branch5x5_1 = ConvBlock(48, 1)
        self.branch5x5_2 = ConvBlock(64, 5, padding='same')
        self.branch3x3dbl_1 = ConvBlock(64, 1)
        self.branch3x3dbl_2 = ConvBlock(96, 3, padding='same')
        self.branch3x3dbl_3 = ConvBlock(96, 3, padding='same')
        self.branch_pool_1 = layers.AvgPool2D(3, 1, padding='same')
        self.branch_pool_2 = ConvBlock(pool_features, 1)

    def call(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = self.branch_pool_1(x)
        branch_pool = self.branch_pool_2(branch_pool)

        outputs = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool])
        return outputs

        
class InceptionB(keras.Model):
    def __init__(self, c7): 
        super(InceptionB, self).__init__()
        self.branch1x1 = ConvBlock(192, 1)
        self.branch7x7_1 = ConvBlock(c7, 1)
        self.branch7x7_2 = ConvBlock(c7, (1,7), zeros=(0,3))
        self.branch7x7_3 = ConvBlock(192, (7,1), zeros=(3,0))
        self.branch7x7dbl_1 = ConvBlock(c7, 1)
        self.branch7x7dbl_2 = ConvBlock(c7, (7,1), zeros=(3,0))
        self.branch7x7dbl_3 = ConvBlock(c7, (1,7), zeros=(0,3))
        self.branch7x7dbl_4 = ConvBlock(c7, (7,1), zeros=(3,0))
        self.branch7x7dbl_5 = ConvBlock(192, (1,7), zeros=(0,3))
        self.branch_pool_1 = layers.AvgPool2D(3, 1, padding='same')
        self.branch_pool_2 = ConvBlock(192, 1)
        
    def call(self, x):
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        branch_pool = self.branch_pool_1(x)
        branch_pool = self.branch_pool_2(branch_pool)

        outputs = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool])
        return outputs

    
class InceptionC(keras.Model): 
    def __init__(self): 
        super(InceptionC, self).__init__()
        self.branch1x1 = ConvBlock(320, 1)
        self.branch3x3_1 = ConvBlock(384, 1)
        self.branch3x3_2a = ConvBlock(384, (1,3), zeros=(0,1))
        self.branch3x3_2b = ConvBlock(384, (3,1), zeros=(1,0))
        self.branch3x3dbl_1 = ConvBlock(448, 1)
        self.branch3x3dbl_2 = ConvBlock(384, 3, padding='same')
        self.branch3x3dbl_3a = ConvBlock(384, (1,3), zeros=(0,1))
        self.branch3x3dbl_3b = ConvBlock(384, (3,1), zeros=(1,0))
        self.branch_pool_1 = layers.AvgPool2D(3, 1, padding='same')
        self.branch_pool_2 = ConvBlock(192, 1)

    def call(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = layers.concatenate([self.branch3x3_2a(branch3x3),
                                        self.branch3x3_2b(branch3x3)])

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = layers.concatenate([self.branch3x3dbl_3a(branch3x3dbl),
                                           self.branch3x3dbl_3b(branch3x3dbl)])
                                          
        branch_pool = self.branch_pool_1(x)
        branch_pool = self.branch_pool_2(branch_pool)

        outputs = layers.concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool])
        return outputs                                    

In [328]:
#下采样 
class ReductionA(keras.Model):
    def __init__(self): 
        super(ReductionA, self).__init__()
        self.branch3x3 = ConvBlock(384, 3, 2)
        self.branch3x3dbl_1 = ConvBlock(64, 1)
        self.branch3x3dbl_2 = ConvBlock(96, 3, padding='same')
        self.branch3x3dbl_3 = ConvBlock(96, 3, 2)
        self.branch_pool_1 = layers.MaxPooling2D(3, 2)

    def call(self, x):
        branch3x3 = self.branch3x3(x)
        
        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = self.branch_pool_1(x)
        
        outputs = layers.concatenate([branch3x3, branch3x3dbl, branch_pool])
        return outputs

    
class ReductionB(keras.Model):
    def __init__(self): 
        super(ReductionB, self).__init__()
        self.branch3x3_1 = ConvBlock(192, 1)
        self.branch3x3_2 = ConvBlock(320, 3, 2)
        self.branch7x7x3_1 = ConvBlock(192, 1)
        self.branch7x7x3_2 = ConvBlock(192, (1,7), zeros=(0,3))
        self.branch7x7x3_3 = ConvBlock(192, (7,1), zeros=(3,0))
        self.branch7x7x3_4 = ConvBlock(192, 3, 2)
        self.branch_pool = layers.MaxPooling2D(3, 2)
        
    def call(self, x):
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch7x7x3 = self.branch7x7x3_1(x)
        branch7x7x3 = self.branch7x7x3_2(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_3(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_4(branch7x7x3)

        branch_pool = self.branch_pool(x)
        outputs = layers.concatenate([branch3x3, branch7x7x3, branch_pool])
        return outputs

In [329]:
class InceptionAux(keras.Model):
    def __init__(self, num_classes): 
        super(InceptionAux, self).__init__()
        self.pool = layers.AvgPool2D(5, 3)
        self.conv0 = ConvBlock(128, 1)
        self.conv1 = ConvBlock(768, 5)
        #gap的话默认输出(1,1)并且压缩只剩下c，而自适应可以自定义output size
        self.adaptive = tfa.layers.AdaptiveAveragePooling2D((1,1)) 
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes)

    def call(self, x): #(b,17,17,768)
        x = self.pool(x) 
        x = self.conv0(x) 
        x = self.conv1(x)
        x = self.adaptive(x) #(b,1,1,768)
        x = self.flatten(x)
        outputs = self.fc(x)
        return outputs

In [319]:
# 检查blocks
inputs = keras.Input(shape=(299,299,3))
model = InceptionA(10)
model.build(input_shape=(None,299,299,3))
model.call(inputs)
model.summary()

<KerasTensor: shape=(None, 299, 299, 234) dtype=float32 (created by layer 'concatenate_15')>

Model: "inception_a_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_549 (ConvBlock)   (None, 299, 299, 64)      512       
_________________________________________________________________
conv_block_550 (ConvBlock)   (None, 299, 299, 48)      384       
_________________________________________________________________
conv_block_551 (ConvBlock)   (None, 299, 299, 64)      77120     
_________________________________________________________________
conv_block_552 (ConvBlock)   (None, 299, 299, 64)      512       
_________________________________________________________________
conv_block_553 (ConvBlock)   (None, 299, 299, 96)      55776     
_________________________________________________________________
conv_block_554 (ConvBlock)   (None, 299, 299, 96)      83424     
_________________________________________________________________
average_pooling2d_94 (Averag (None, 299, 299, 3)    

In [334]:
class InceptionV3(keras.Model):
    def __init__(self, num_classes=1000, aux_logits=True): 
        super(InceptionV3, self).__init__(name='inceptionv3')
        self.aux_logits = aux_logits
        # Stem
        self.Conv2d_1a_3x3 = ConvBlock(32, 3, 2)
        self.Conv2d_2a_3x3 = ConvBlock(32, 3)
        self.Conv2d_2b_3x3 = ConvBlock(64, 3, padding='same')
        self.maxpool1 = layers.MaxPooling2D(3, 2)
        self.Conv2d_3b_1x1 = ConvBlock(80, 1)
        self.Conv2d_4a_3x3 = ConvBlock(192, 3)
        self.maxpool2 = layers.MaxPooling2D(3, 2)
        
        # Inception modules
        self.Mixed_5b = InceptionA(32)
        self.Mixed_5c = InceptionA(64)
        self.Mixed_5d = InceptionA(64)
        self.Mixed_6a = ReductionA()
        self.Mixed_6b = InceptionB(128)
        self.Mixed_6c = InceptionB(160)
        self.Mixed_6d = InceptionB(160)
        self.Mixed_6e = InceptionB(192)

        if self.aux_logits: # Auxiliary classifier
            self.AuxLogits = InceptionAux(num_classes)
        self.Mixed_7a = ReductionB()
        self.Mixed_7b = InceptionC()
        self.Mixed_7c = InceptionC()
        self.avgpool = tfa.layers.AdaptiveAveragePooling2D((1,1))
        self.dropout = layers.Dropout(0.3)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes)
        
    def call(self, x, training=False): #(b, 299, 299, 3)
        x = self.Conv2d_1a_3x3(x)
        x = self.Conv2d_2a_3x3(x)
        x = self.Conv2d_2b_3x3(x)
        x = self.maxpool1(x)
        x = self.Conv2d_3b_1x1(x)
        x = self.Conv2d_4a_3x3(x)
        x = self.maxpool2(x)
        
        x = self.Mixed_5b(x)
        x = self.Mixed_5c(x)
        x = self.Mixed_5d(x)
        x = self.Mixed_6a(x)
        x = self.Mixed_6b(x)
        x = self.Mixed_6c(x)
        x = self.Mixed_6d(x)
        x = self.Mixed_6e(x)
        
        #只在训练时候使用，model.fit时自动调整为True，预测不用
        if training and self.aux_logits:
            aux = self.AuxLogits(x)
        else:
            aux = None
        
        x = self.Mixed_7a(x)
        x = self.Mixed_7b(x)
        x = self.Mixed_7c(x)
        x = self.avgpool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x, aux

In [336]:
inputs = keras.Input(shape=(299,299,3))
model = InceptionV3()
model.build(input_shape=(None,299,299,3))
model.call(inputs, training=True)
model.summary()

(<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'dense_68')>,
 <KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'inception_aux_11')>)

Model: "inceptionv3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_966 (ConvBlock)   (None, 149, 149, 32)      1024      
_________________________________________________________________
conv_block_967 (ConvBlock)   (None, 147, 147, 32)      9376      
_________________________________________________________________
conv_block_968 (ConvBlock)   (None, 147, 147, 64)      18752     
_________________________________________________________________
max_pooling2d_102 (MaxPoolin (None, 73, 73, 64)        0         
_________________________________________________________________
conv_block_969 (ConvBlock)   (None, 73, 73, 80)        5520      
_________________________________________________________________
conv_block_970 (ConvBlock)   (None, 71, 71, 192)       139200    
_________________________________________________________________
max_pooling2d_103 (MaxPoolin (None, 35, 35, 192)       

In [26]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train, validation_data=test, epochs=3)

Epoch 1/3
938/938 [==============================] - 31s 32ms/step - loss: 0.9166 - accuracy: 0.7087 - val_loss: 0.2903 - val_accuracy: 0.9144
Epoch 2/3
938/938 [==============================] - 32s 34ms/step - loss: 0.2362 - accuracy: 0.9287 - val_loss: 0.1853 - val_accuracy: 0.9423
Epoch 3/3
938/938 [==============================] - 32s 34ms/step - loss: 0.1580 - accuracy: 0.9517 - val_loss: 0.1220 - val_accuracy: 0.9624


#### InceptionV4
Paper: [Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning](https://arxiv.org/abs/1602.07261)

<img src='https://miro.medium.com/max/1000/1*15sIUNOxqVyDPmGGqefyVw.png'></img>

#### Inception ResNet
Paper: [Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning](https://arxiv.org/abs/1602.07261)

<img src='https://miro.medium.com/max/1000/1*xpb6QFQ4IknSmxmgai8w-Q.png'></img>

#### Pretrained

In [188]:
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2

In [207]:
model = InceptionV3(input_shape=(299,299,3), include_top=True,
                    weights=None, classes=1000) 
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 149, 149, 32) 864         input_24[0][0]                   
__________________________________________________________________________________________________
batch_normalization_192 (BatchN (None, 149, 149, 32) 96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 149, 149, 32) 0           batch_normalization_192[0][0]    
_______________________________________________________________________________________